<a href="https://colab.research.google.com/github/SiddhiMarri/PostureDetection/blob/main/detectkeypoints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics
!pip install opencv-python
!pip install opencv-python-headless

In [ ]:
import cv2
from ultralytics import YOLO
import torch

# Paths to the video file and YOLO model
video_path = "/content/WhatsApp Video 2024-11-22 at 15.10.15.mp4"  # Replace with your input video file name

In [ ]:
import cv2
from ultralytics import YOLO
import torch

# Load the pre-trained YOLOv8n-pose model
model = YOLO('best (1).pt')

# Open the video file
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
output_filename = "annotated_output.mp4"

# Define codec and create VideoWriter for the output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_filename, fourcc, fps, (frame_width, frame_height))

# File to save keypoints and bounding box data
results_file = "detection_results.txt"
with open(results_file, "w") as f:
    f.write("Frame, Keypoints, Bounding Boxes, Confidence Scores, Classes\n")

# Process the video frame by frame
frame_index = 0
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of video. Exiting...")
            break

        # Perform inference
        results = model(frame)

        # Process results for this frame
        for result in results:
            boxes = result.boxes  # Bounding boxes
            keypoints = result.keypoints  # Keypoints
            for i, box in enumerate(boxes):
                bbox = box.xyxy[0].tolist()  # Bounding box coordinates
                confidence = box.conf[0].item()  # Confidence score
                cls = box.cls[0].item()  # Class label

                # Extract keypoints for this box
                if keypoints is not None:
                    kp = keypoints[i].xy.cpu().numpy().tolist()  # Keypoints for this instance
                else:
                    kp = "N/A"

                # Write results to the text file
                with open(results_file, "a") as f:
                    f.write(f"{frame_index}, {kp}, {bbox}, {confidence}, {cls}\n")

        # Render results on the frame
        annotated_frame = result.plot()

        # Save the processed frame to the output video
        out.write(annotated_frame)

        frame_index += 1

except Exception as e:
    print(f"Error during video processing: {e}")

finally:
    # Release resources
    cap.release()
    out.release()
    print(f"Output video saved as {output_filename}.")
    print(f"Detection results saved in {results_file}.")

# Download the output video and results file
from google.colab import files
files.download(output_filename)
files.download(results_file)


0: 384x640 5 persons, 10.7ms
Speed: 1.8ms preprocess, 10.7ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 10.4ms
Speed: 2.0ms preprocess, 10.4ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 11.4ms
Speed: 2.1ms preprocess, 11.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 11.4ms
Speed: 2.0ms preprocess, 11.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 8.5ms
Speed: 2.0ms preprocess, 8.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 10.0ms
Speed: 2.0ms preprocess, 10.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 9.0ms
Speed: 2.4ms preprocess, 9.0ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 11.4ms
Speed: 2.0ms preprocess, 11.4ms inference, 1.8ms postprocess per image at shape (1, 3

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import ast

# Path to the results file
results_file = '/content/detection_results.txt'

# Initialize a dictionary to store keypoints by frame
keypoints_by_frame = {}

# Read and process the results file
with open(results_file, 'r') as file:
    # Skip the header line
    next(file)
    for line in file:
        # Use the first comma split to separate the frame from the rest of the line
        frame_split = line.split(',', 1)
        frame = int(frame_split[0].strip())
        rest = frame_split[1].strip()

        # Extract keypoints, bbox, confidence, and class
        try:
            # Parse the remaining line as a list of values
            data = ast.literal_eval(rest)

            keypoints = data[0]  # Keypoints (list of [x, y] coordinates)
            bbox = data[1]  # Bounding box [x1, y1, x2, y2]
            confidence = data[2]  # Confidence score
            cls = data[3]  # Class label

            # Store the data by frame
            if frame not in keypoints_by_frame:
                keypoints_by_frame[frame] = []
            keypoints_by_frame[frame].append({
                'keypoints': keypoints,
                'bbox': bbox,
                'confidence': confidence,
                'class': cls
            })
        except (ValueError, SyntaxError) as e:
            print(f"Error parsing line: {line}")
            print(f"Exception: {e}")
            continue

# Example: Access keypoints for frame 0
frame_0_keypoints = keypoints_by_frame.get(0, [])
for detection in frame_0_keypoints:
    print(f"Bounding Box: {detection['bbox']}")
    print(f"Confidence: {detection['confidence']}")
    print(f"Class: {detection['class']}")
    print("Keypoints:")
    for idx, (x, y) in enumerate(detection['keypoints'][0]):
        print(f"  Keypoint {idx + 1}: ({x}, {y})")


Bounding Box: [368.0, 6.0, 1083.0, 711.0]
Confidence: 0.9041202664375305
Class: 0.0
Keypoints:
  Keypoint 1: (709.6782836914062, 128.88262939453125)
  Keypoint 2: (754.0595703125, 90.65399169921875)
  Keypoint 3: (659.9464721679688, 94.8695068359375)
  Keypoint 4: (813.4784545898438, 137.56317138671875)
  Keypoint 5: (600.2918090820312, 149.66458129882812)
  Keypoint 6: (943.9601440429688, 411.87664794921875)
  Keypoint 7: (487.17047119140625, 414.8376159667969)
  Keypoint 8: (0.0, 0.0)
  Keypoint 9: (0.0, 0.0)
  Keypoint 10: (0.0, 0.0)
  Keypoint 11: (0.0, 0.0)
  Keypoint 12: (0.0, 0.0)
  Keypoint 13: (0.0, 0.0)
  Keypoint 14: (0.0, 0.0)
  Keypoint 15: (0.0, 0.0)
  Keypoint 16: (0.0, 0.0)
  Keypoint 17: (0.0, 0.0)
Bounding Box: [1164.0, 14.0, 1280.0, 708.0]
Confidence: 0.7736448645591736
Class: 0.0
Keypoints:
  Keypoint 1: (0.0, 0.0)
  Keypoint 2: (0.0, 0.0)
  Keypoint 3: (0.0, 0.0)
  Keypoint 4: (0.0, 0.0)
  Keypoint 5: (0.0, 0.0)
  Keypoint 6: (0.0, 0.0)
  Keypoint 7: (1229.37255859